In [1]:
import pandas as pd

In [7]:
radio_topics=pd.read_csv('radio_sample_topics.csv')
radio_read=pd.read_csv('readability_radio_results_final.csv')
radio_grammar=pd.read_csv('grammar_radio.csv')
radio_sentiment=pd.read_csv('stratified_radio_sample_TextBlob_HYBRID.csv')


In [9]:
import pandas as pd

# === Load Data ===
radio_topics = pd.read_csv('radio_sample_topics.csv')
radio_read = pd.read_csv('readability_radio_results_final.csv')
radio_grammar = pd.read_csv('grammar_radio.csv')
radio_sentiment = pd.read_csv('stratified_radio_sample_TextBlob_HYBRID.csv')

# === General-purpose Safe Merge Function ===
def safe_merge_all(dfs, on=["text", "station", "range_key"], how="inner"):
    # Step 1: Clean and standardize merge keys
    for df in dfs:
        if "text" in on:
            df["text"] = df["text"].astype(str).str.strip().str.lower()
        if "station" in on:
            df["station"] = df["station"].astype(str).str.strip().str.lower()
        if "range_key" in on:
            df["range_key"] = pd.to_datetime(df["range_key"], errors='coerce')

    # Step 2: Merge sequentially, dropping overlapping columns
    merged = dfs[0]
    for i, df in enumerate(dfs[1:], start=2):
        # Drop duplicate columns (excluding the keys)
        duplicate_cols = set(merged.columns).intersection(df.columns) - set(on)
        df_clean = df.drop(columns=duplicate_cols)
        
        # Report duplicates
        dup_count = df.duplicated(subset=on).sum()
        print(f"DF{i}: {dup_count} duplicate rows based on keys {on}")

        merged = pd.merge(merged, df_clean, on=on, how=how)
        print(f"After merging DF{i}: {len(merged)} rows")

    return merged

# === Merge All Four ===
radio_merged = safe_merge_all([radio_topics, radio_grammar, radio_read, radio_sentiment])

# === Final Output ===
print("✅ Final merged shape:", radio_merged.shape)
print(radio_merged.head())

# Optional: Save to CSV
radio_merged.to_csv("radio_merged_final.csv", index=False)


DF2: 0 duplicate rows based on keys ['text', 'station', 'range_key']
After merging DF2: 624 rows
DF3: 0 duplicate rows based on keys ['text', 'station', 'range_key']
After merging DF3: 624 rows
DF4: 0 duplicate rows based on keys ['text', 'station', 'range_key']
After merging DF4: 624 rows
✅ Final merged shape: (624, 43)
  station           range_key  \
0    kawc 2024-11-30 09:00:00   
1    kawc 2024-09-26 06:00:00   
2    kvoi 2024-11-15 15:00:00   
3    kqna 2024-11-03 21:00:00   
4    kvoi 2024-09-01 00:00:00   

                                                text state  abortion  \
0  weather forecast for yuma today is mostly sunn...    AZ  0.000305   
1  k-a-w-c.classical music of peter van de graaff...    AZ  0.000393   
2  at cortero, tucson's only live local news and ...    AZ  0.000520   
3  can't get enough of ycca's hammer time show?wa...    AZ  0.000351   
4  under the gaza city of rafa, including that of...    AZ  0.000654   

   international affairs  immigration   econo

In [11]:
radio_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   station                624 non-null    object        
 1   range_key              624 non-null    datetime64[ns]
 2   text                   624 non-null    object        
 3   state                  624 non-null    object        
 4   abortion               624 non-null    float64       
 5   international affairs  624 non-null    float64       
 6   immigration            624 non-null    float64       
 7   economy                624 non-null    float64       
 8   violent crime          624 non-null    float64       
 9   climate change         624 non-null    float64       
 10  common                 624 non-null    int64         
 11  proper                 624 non-null    int64         
 12  proper_to_common       624 non-null    float64       
 13  prope

In [12]:
merged_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35098 entries, 0 to 35097
Data columns (total 48 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   id                                             35098 non-null  object 
 1   attributes.search_data_fields.channel_name     35057 non-null  object 
 2   channel_id                                     35098 non-null  int64  
 3   attributes.created_at                          35098 non-null  object 
 4   attributes.post_type                           35098 non-null  object 
 5   attributes.search_data_fields.platform_name    35098 non-null  object 
 6   attributes.description                         34039 non-null  object 
 7   attributes.search_data_fields.all_text         34499 non-null  object 
 8   attributes.search_data_fields.transcript_text  35098 non-null  object 
 9   actor_attributes.Post-Election Status          870

regression for sentiment

In [19]:
import pandas as pd
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

# === Load data ===
radio_df = pd.read_csv("radio_merged_final.csv")
media_df = pd.read_csv("merged_social_all.csv")

# Simple composite sentiment score
radio_df['composite_sentiment'] = radio_df['positive_prop'] - radio_df['negative_prop']

# === Convert to categorical for social media dataset ===
cat_vars = [
    'actor_attributes.Gender',
    'actor_attributes.Race',
    'actor_attributes.Registered Political Party',
    'actor_attributes.Body Name',
    'actor_attributes.State',
    'attributes.search_data_fields.platform_name'
]
for col in cat_vars:
    media_df[col] = media_df[col].astype("category")

# === Define Social Media Formula with Q() ===
media_formula = """
sentiment_strength ~
    abortion + Q('international affairs') + immigration + economy + Q('violent crime') + Q('climate change') +
    svo_count + proper_to_common + gunning_fog +
    cnt_president_biden + cnt_president_trump +
    C(Q('actor_attributes.Gender')) +
    C(Q('actor_attributes.Race')) +
    C(Q('actor_attributes.Registered Political Party')) +
    C(Q('actor_attributes.Body Name')) +
    C(Q('actor_attributes.State')) +
    C(Q('attributes.search_data_fields.platform_name'))
"""

# === Define Radio Formula with Q() ===
radio_formula = """
composite_sentiment ~
    abortion + Q('international affairs') + immigration + economy + Q('violent crime') + Q('climate change') +
    svo_count + proper_to_common + smog_index +
    cnt_president_biden + cnt_president_trump
"""

# === Fit models ===
media_model = smf.ols(formula=media_formula, data=media_df).fit()
radio_model = smf.ols(formula=radio_formula, data=radio_df).fit()

# === Stargazer summary ===
stargazer = Stargazer([radio_model, media_model])
stargazer.title("Regression Results: Radio vs. Social Media Sentiment")
stargazer.custom_columns(["Radio", "Social Media"], [1, 1])

# === Render or save the HTML output ===
html_output = stargazer.render_html()

# Optional: display in notebook or save to file
from IPython.core.display import display, HTML
display(HTML(html_output))

with open("radio_vs_social_sentiment_regression.html", "w") as f:
    f.write(html_output)

# Instead of render_html()
latex_output = stargazer.render_latex()

# Save to .tex file
with open("regression_table.tex", "w") as f:
    f.write(latex_output)

/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_15890/2504637552.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  media_df = pd.read_csv("merged_social_all.csv")
/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_15890/2504637552.py:59: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


regression for topics

In [7]:
import pandas as pd
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

# === Load data ===
radio_df = pd.read_csv("radio_merged_final.csv")
media_df = pd.read_csv("merged_social_all.csv")

# Simple composite sentiment score
radio_df['composite_sentiment'] = radio_df['positive_prop'] - radio_df['negative_prop']

# === Convert to categorical for social media dataset ===
cat_vars = [
    'actor_attributes.Gender',
    'actor_attributes.Race',
    'actor_attributes.Registered Political Party',
    'actor_attributes.Body Name',
    'actor_attributes.State',
    'attributes.search_data_fields.platform_name'
]
for col in cat_vars:
    media_df[col] = media_df[col].astype("category")

# === Define Social Media Formula with Q() ===
media_formula = """
Q('climate change') ~
    svo_count + proper_to_common + gunning_fog +
    cnt_president_biden + cnt_president_trump + sentiment_strength+
    C(Q('actor_attributes.Gender')) +
    C(Q('actor_attributes.Race')) +
    C(Q('actor_attributes.Registered Political Party')) +
    C(Q('actor_attributes.Body Name')) +
    C(Q('actor_attributes.State')) +
    C(Q('attributes.search_data_fields.platform_name'))
"""

# === Define Radio Formula with Q() ===
radio_formula = """
Q('climate change') ~
    svo_count + proper_to_common + smog_index +composite_sentiment+
    cnt_president_biden + cnt_president_trump
"""

# === Fit models ===
media_model = smf.ols(formula=media_formula, data=media_df).fit()
radio_model = smf.ols(formula=radio_formula, data=radio_df).fit()

# === Stargazer summary ===
stargazer = Stargazer([radio_model, media_model])
stargazer.title("Regression Results: Radio vs. Social Media Sentiment")
stargazer.custom_columns(["Radio", "Social Media"], [1, 1])

# === Render or save the HTML output ===
html_output = stargazer.render_html()

# Optional: display in notebook or save to file
from IPython.core.display import display, HTML
display(HTML(html_output))

with open("radio_vs_social_sentiment_regression.html", "w") as f:
    f.write(html_output)

# Instead of render_html()
latex_output = stargazer.render_latex()

# Save to .tex file
with open("regression_table.tex", "w") as f:
    f.write(latex_output)

/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_23036/1442621078.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  media_df = pd.read_csv("merged_social_all.csv")
/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_23036/1442621078.py:57: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [19]:
social_topics=pd.read_csv('social_scored_output.csv')
social_read=pd.read_csv('readability_junkipedia_results.csv')
social_grammar=pd.read_csv('grammar_junk 2.csv')
social_sentiment=pd.read_csv('socialMedia_with_advanced_sentiment.csv')

/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_11276/1983325635.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_topics=pd.read_csv('social_scored_output.csv')
/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_11276/1983325635.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_read=pd.read_csv('readability_junkipedia_results.csv')
/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_11276/1983325635.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_grammar=pd.read_csv('grammar_junk 2.csv')
/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_11276/1983325635.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_sentiment=pd.read_csv('socialMedia_with_advanced_sentiment.csv')


In [8]:
# First merge: topics + read
merged_social = pd.merge(
    social_topics, social_read,
    on='id',
    how='inner',  # or 'outer'/'left' depending on what you want
    suffixes=('', '_read')  # adds _read to overlapping columns from social_read
)

# Second merge: + grammar
merged_social = pd.merge(
    merged_social, social_grammar,
    on='id',
    how='inner',
    suffixes=('', '_grammar')  # adds _grammar to overlapping columns from social_grammar
)

# Transpose to find duplicate columns by content
# Drop all columns ending in _read or _grammar (common pattern for duplicates)
merged_social = merged_social.drop(columns=[col for col in merged_social.columns if col.endswith('_read') or col.endswith('_grammar')])



# Preview
print(merged_social.head())

          id attributes.search_data_fields.channel_name  channel_id  \
0  273761856                              Kathleen Winn    25509047   
1  288860713               Congressman Paul Gosar 🇺🇸 ✝️    26148902   
2  186999089                                Abe Hamadeh     8056353   
3  263685950                                Abe Hamadeh     8056353   
4  408635753                               Debbie Lesko     9588349   

      attributes.created_at attributes.post_type  \
0  2024-07-19T00:14:47.461Z            ['photo']   
1  2024-08-08T04:12:03.332Z            ['video']   
2  2024-01-12T12:26:21.619Z   ['photo', 'photo']   
3  2024-07-09T07:41:04.917Z            ['video']   
4  2025-01-10T03:38:38.883Z                   []   

  attributes.search_data_fields.platform_name  \
0                             InstagramDirect   
1                                    Telegram   
2                                     Twitter   
3                                     Twitter   
4              

In [2]:
import pandas as pd

# Step 1: Load datasets
social_topics = pd.read_csv('social_scored_output.csv')
social_read = pd.read_csv('readability_junkipedia_results.csv')
social_grammar = pd.read_csv('grammar_junk 2.csv')
social_sentiment = pd.read_csv('socialMedia_with_advanced_sentiment.csv')

# Step 2: Ensure consistent 'id' format
for df in [social_topics, social_read, social_grammar, social_sentiment]:
    df['id'] = df['id'].astype(str).str.strip()

# Step 3: Helper function to drop duplicated columns
def drop_duplicate_columns(df_main, df_to_merge, key='id'):
    shared_cols = set(df_main.columns).intersection(df_to_merge.columns) - {key}
    return df_to_merge.drop(columns=shared_cols)

# Step 4: Merge in sequence
df1 = social_topics
df2 = drop_duplicate_columns(df1, social_read)
merged = pd.merge(df1, df2, on='id', how='inner')

df3 = drop_duplicate_columns(merged, social_grammar)
merged = pd.merge(merged, df3, on='id', how='inner')

df4 = drop_duplicate_columns(merged, social_sentiment)
merged_final = pd.merge(merged, df4, on='id', how='inner')

# Step 5: Final check
print(f"Final merged shape: {merged_final.shape}")
print(merged_final.head())

# Optional: Save to file
# merged_final.to_csv("merged_social_all.csv", index=False)


/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_15890/4242361631.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_topics = pd.read_csv('social_scored_output.csv')
/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_15890/4242361631.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_read = pd.read_csv('readability_junkipedia_results.csv')
/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_15890/4242361631.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_grammar = pd.read_csv('grammar_junk 2.csv')


Final merged shape: (35098, 47)
          id attributes.search_data_fields.channel_name  channel_id  \
0  273761856                              Kathleen Winn    25509047   
1  288860713               Congressman Paul Gosar 🇺🇸 ✝️    26148902   
2  186999089                                Abe Hamadeh     8056353   
3  263685950                                Abe Hamadeh     8056353   
4  408635753                               Debbie Lesko     9588349   

      attributes.created_at attributes.post_type  \
0  2024-07-19T00:14:47.461Z            ['photo']   
1  2024-08-08T04:12:03.332Z            ['video']   
2  2024-01-12T12:26:21.619Z   ['photo', 'photo']   
3  2024-07-09T07:41:04.917Z            ['video']   
4  2025-01-10T03:38:38.883Z                   []   

  attributes.search_data_fields.platform_name  \
0                             InstagramDirect   
1                                    Telegram   
2                                     Twitter   
3                               

/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_15890/4242361631.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_sentiment = pd.read_csv('socialMedia_with_advanced_sentiment.csv')


In [5]:
# Create a sentiment strength score: +score for positive, 0 for neutral, -score for negative
def compute_sentiment_strength(row):
    if row['roberta_label'] == 'LABEL_2':    # positive
        return row['roberta_score']
    elif row['roberta_label'] == 'LABEL_0':  # negative
        return -row['roberta_score']
    else:  # LABEL_1 (neutral)
        return 0

merged_final['sentiment_strength'] = merged_final.apply(compute_sentiment_strength, axis=1)
merged_final.to_csv("merged_social_all.csv", index=False)

In [10]:
column_names = merged_social.columns.tolist()
column_names

['id',
 'attributes.search_data_fields.channel_name',
 'channel_id',
 'attributes.created_at',
 'attributes.post_type',
 'attributes.search_data_fields.platform_name',
 'attributes.description',
 'attributes.search_data_fields.all_text',
 'attributes.search_data_fields.transcript_text',
 'actor_attributes.Post-Election Status',
 'actor_attributes.Incumbent',
 'actor_attributes.Gender',
 'actor_attributes.Race',
 'actor_attributes.Registered Political Party',
 'actor_attributes.Body Name',
 'actor_attributes.State',
 'actor_attributes.Electoral District',
 'name',
 'clean_text',
 'abortion',
 'international affairs',
 'immigration',
 'economy',
 'violent crime',
 'climate change',
 'gunning_fog',
 'common',
 'proper',
 'proper_to_common',
 'proper_prop',
 'cnt_president_trump',
 'cnt_president_biden',
 'cnt_state_self',
 'svo_count',
 'svo_tbh_count',
 'subj_donald_trump',
 'obj_donald_trump',
 'subj_joe_biden',
 'obj_joe_biden',
 'subj_kamala_harris',
 'obj_kamala_harris']

In [13]:
print(df['actor_attributes.Registered Political Party'].unique())
print(df['actor_attributes.Registered Political Party'].dtype)


['Republican Party' 'Democratic Party']
object


In [30]:
social_sentiment=pd.read_csv('socialMedia_with_advanced_sentiment.csv')

/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_34470/3891951130.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  social_sentiment=pd.read_csv('socialMedia_with_advanced_sentiment.csv')


In [32]:
# Merge on 'id'
merged_social = pd.merge(
    merged_social,
    social_sentiment,
    on='id',
    how='left',  # or 'inner' if you only want overlapping ids
    suffixes=('', '_sentiment')
)

# Drop exact duplicate columns (with identical content)

# Drop all columns ending in _read or _grammar (common pattern for duplicates)
merged_social = merged_social.drop(columns=[col for col in merged_social.columns if col.endswith('_sentiment')])


In [33]:
# Create a sentiment strength score: +score for positive, 0 for neutral, -score for negative
def compute_sentiment_strength(row):
    if row['roberta_label'] == 'LABEL_2':    # positive
        return row['roberta_score']
    elif row['roberta_label'] == 'LABEL_0':  # negative
        return -row['roberta_score']
    else:  # LABEL_1 (neutral)
        return 0

merged_social['sentiment_strength'] = merged_social.apply(compute_sentiment_strength, axis=1)


In [34]:
# Map label strings for clarity (optional, for use as categorical predictor)
merged_social['roberta_sentiment'] = merged_social['roberta_label'].map({
    'LABEL_2': 'positive',
    'LABEL_1': 'neutral',
    'LABEL_0': 'negative'
})
